# side model feature selection

In [9]:
import pandas as pd

df_features = pd.read_parquet("data/features_240m.parquet")
print(df_features.shape)
df_features.isna().sum(axis=0).sort_values(ascending=False)

(11470, 15051)


720m_dft_dom_cycle_ddt_lag29    237
720m_dft_dom_cycle_dt_lag29     234
720m_dft_dom_cycle_ddt_lag28    234
720m_dft_dom_cycle_dt_lag28     231
720m_dft_dom_cycle_lag29        231
                               ... 
240m_conv_19                      0
240m_conv_20                      0
240m_conv_21                      0
240m_adaptive_cci                 0
240m_conv_10                      0
Length: 15051, dtype: int64

In [10]:
import numpy as np
from jesse import utils

label = np.load("data/label_side.npy")
df_label = utils.numpy_candles_to_dataframe(label[:, :6]).set_index("date")
df_label["side_label"] = label[:, 6]
df_label.head(1)

,open,close,high,low,volume,side_label
date,,,,,,
2020-01-01 22:41:00,7195.86,7353.0,7407.28,6863.44,298961.076,1.0


In [11]:
BAD_FRONT = 237

side_features = df_features.iloc[BAD_FRONT:]
side_label = df_label.iloc[BAD_FRONT:]

side_label["side_label"] = (side_label["side_label"].astype(int) == 1).astype(int)
side_label = side_label["side_label"].to_numpy()

print(side_features.shape)
print(side_label.shape)

side_features.isna().sum(axis=0).sort_values(ascending=False)

(11233, 15051)
(11233,)


/var/folders/2g/md1t2j7s6hjdjc5jlvkmn7_c0000gn/T/ipykernel_72435/2898252471.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  side_label["side_label"] = (side_label["side_label"].astype(int) == 1).astype(int)


720m_adx_7                   0
80m_adx_7_lag2               0
240m_williams_r_ddt_lag22    0
240m_williams_r_ddt_lag23    0
240m_williams_r_ddt_lag24    0
                            ..
240m_adx_7_lag2              0
240m_adx_7_lag3              0
240m_adx_7_lag4              0
240m_adx_7_lag5              0
80m_williams_r_ddt_lag29     0
Length: 15051, dtype: int64

In [13]:
from custom_indicators.toolbox.feature_selection.rfcq_selector import RFCQSelector

selector = RFCQSelector(max_features=2500)
selector.fit(side_features, side_label)

➤ 识别数值型变量...
➤ 计算特征与目标变量的相关性(使用随机森林)...
✓ 选择第1个特征: 240m_vwap_dt (最大重要性: 0.0449)
➤ 计算特征冗余度...
➤ 总计选择2500个特征 (已选择1个，还需选择2499个)...
➤ 开始MRMR迭代选择过程...


选择特征: 100%|███████████████████████████████████████████████| 2499/2499 [08:56<00:00,  4.66特征/s]



✅ 特征选择完成：从15051个特征中选择了2500个，舍弃了12551个
✅ 选择的特征: ['240m_vwap_dt', '240m_trendflex_lag20', '240m_ac_0', '240m_acc_swing_index_ddt', '80m_stc_ddt_lag3', '240m_cmma', '240m_fisher_ddt', '240m_adx_7_dt', '240m_evenbetter_sinewave_short_dt', '80m_vwap_dt', '240m_williams_r_dt', '240m_fisher_dt', '240m_cmma_dt', '240m_williams_r_lag1', '240m_acc_swing_index_lag4', '240m_ehlers_early_onset_trend_ddt', '240m_williams_r', '80m_stc_dt_lag1', '240m_norm_on_balance_volume_dt', '80m_vwap_dt_lag1', '240m_acc_swing_index_dt', '80m_ehlers_early_onset_trend_ddt', '720m_forecast_oscillator', '240m_natr_dt', '80m_cmma', '720m_voss_lag13', '240m_stc_dt', '240m_vwap_dt_lag2', '240m_adx_7_ddt', '240m_dft_dom_cycle_ddt_lag29', '80m_stc_ddt', '240m_roofing_filter_ddt', '80m_vwap_dt_lag3', '240m_adx_14_dt', '240m_aroon_diff_dt', '80m_ehlers_early_onset_trend_ddt_lag1', '240m_reactivity', '80m_evenbetter_sinewave_short_dt', '240m_forecast_oscillator', '80m_norm_on_balance_volume', '240m_adx_14_ddt_lag12', '80m_s

In [14]:
side_res = pd.Series(selector.relevance_, index=selector.variables_).sort_values(
    ascending=False
)
side_res

240m_vwap_dt                0.044896
240m_cmma                   0.030532
240m_williams_r_dt          0.023104
240m_williams_r_lag1        0.023010
240m_fisher_dt              0.022916
                              ...   
240m_adx_7_ddt_lag25        0.000000
240m_adx_7_ddt_lag26        0.000000
240m_adx_7_ddt_lag27        0.000000
240m_adx_7_ddt_lag28        0.000000
80m_williams_r_ddt_lag29    0.000000
Length: 15051, dtype: float64

In [15]:
import json

with open(f"data/side_features_{selector.__class__.__name__}.json", "w") as f:
    json.dump(side_res.to_dict(), f, indent=4)

In [1]:
import json

import pandas as pd

with open("data/side_features_RFCQSelector.json", "r") as f:
    side_res = pd.Series(json.load(f))

In [2]:
from strategies.BinanceMLV2.config import (
    DOLLAR_BAR_SHORT_TERM,
    DOLLAR_BAR_MID_TERM,
    DOLLAR_BAR_LONG_TERM,
)

feature_info = {
    "side": {
        DOLLAR_BAR_SHORT_TERM: [],
        DOLLAR_BAR_MID_TERM: [],
        DOLLAR_BAR_LONG_TERM: [],
    },
    "meta": {
        DOLLAR_BAR_SHORT_TERM: [],
        DOLLAR_BAR_MID_TERM: [],
        DOLLAR_BAR_LONG_TERM: [],
        "model_res": [],
    },
}

for k, v in side_res[side_res > 0].to_dict().items():
    if k.startswith(f"{DOLLAR_BAR_SHORT_TERM}_"):
        feature_info["side"][DOLLAR_BAR_SHORT_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_MID_TERM}_"):
        feature_info["side"][DOLLAR_BAR_MID_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_LONG_TERM}_"):
        feature_info["side"][DOLLAR_BAR_LONG_TERM].append(k)
    else:
        raise ValueError(f"Unknown feature: {k}")


feature_info["side"][DOLLAR_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_SHORT_TERM])),
)
feature_info["side"][DOLLAR_BAR_MID_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_MID_TERM])),
)
feature_info["side"][DOLLAR_BAR_LONG_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_LONG_TERM])),
)


feature_info["meta"][DOLLAR_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_SHORT_TERM])),
)
feature_info["meta"][DOLLAR_BAR_MID_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_MID_TERM])),
)
feature_info["meta"][DOLLAR_BAR_LONG_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_LONG_TERM])),
)
feature_info["meta"]["model_res"] = sorted(
    list(set(feature_info["meta"]["model_res"])),
)

with open("strategies/BinanceMLV2/feature_info.json", "w") as f:
    json.dump(feature_info, f, indent=4)

# meta model feature selection

In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from strategies.BinanceMLV2.config import SIDE_ALL

df_features = pd.read_parquet("data/features_240m.parquet")
meta_label = np.load("data/label_meta.npy")

side_model = lgb.Booster(model_file="strategies/BinanceMLV2/model/model_side.txt")
side_model_pred_prob = side_model.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_pred_prob

assert df_features.shape[0] == meta_label.shape[0]
assert "side_model_res" in df_features.columns

df_features.isna().sum(axis=0).sort_values(ascending=False)

720m_dft_dom_cycle_ddt_lag29    237
720m_dft_dom_cycle_dt_lag29     234
720m_dft_dom_cycle_ddt_lag28    234
720m_dft_dom_cycle_dt_lag28     231
720m_dft_dom_cycle_lag29        231
                               ... 
240m_comb_spectrum_pwr_14         0
240m_comb_spectrum_pwr_13         0
240m_comb_spectrum_pwr_11         0
240m_comb_spectrum_pwr_10         0
side_model_res                    0
Length: 15052, dtype: int64

In [2]:
BAD_FRONT = 237

meta_features = df_features[BAD_FRONT:]
meta_label = meta_label[BAD_FRONT:]

meta_label = pd.Series(meta_label[:, 6].astype(int), index=meta_features.index)

# meta_features = meta_features[meta_label["ret"].notna()]
# meta_label = meta_label[meta_label["ret"].notna()]["bin"]

print(meta_features.shape)
print(meta_label.shape)

meta_features.isna().sum(axis=0).sort_values(ascending=False)

(11233, 15052)
(11233,)


720m_adx_7                   0
80m_adx_7_lag3               0
240m_williams_r_ddt_lag23    0
240m_williams_r_ddt_lag24    0
240m_williams_r_ddt_lag25    0
                            ..
240m_adx_7_lag2              0
240m_adx_7_lag3              0
240m_adx_7_lag4              0
240m_adx_7_lag5              0
side_model_res               0
Length: 15052, dtype: int64

In [3]:
# meta feature selection
from custom_indicators.toolbox.feature_selection.rfcq_selector import RFCQSelector

selector = RFCQSelector(max_features=2500)
selector.fit(meta_features, meta_label)
meta_res = pd.Series(selector.relevance_, index=selector.variables_).sort_values(
    ascending=False
)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


➤ 识别数值型变量...
➤ 计算特征与目标变量的相关性(使用随机森林)...
✓ 选择第1个特征: 240m_adx_14_ddt (最大重要性: 0.0200)
➤ 计算特征冗余度...
➤ 总计选择2500个特征 (已选择1个，还需选择2499个)...
➤ 开始MRMR迭代选择过程...


选择特征: 100%|███████████████████████████████████████████████| 2499/2499 [08:32<00:00,  4.88特征/s]



✅ 特征选择完成：从15052个特征中选择了2500个，舍弃了12552个
✅ 选择的特征: ['240m_adx_14_ddt', '80m_vwap_lag7', '240m_roll_impact_dt_lag5', '240m_price_change_oscillator_ddt', '80m_stc_lag24', '240m_acp_pwr_4', '720m_roll_measure_dt_lag10', '720m_hasbrouck_lambda_dt_lag14', '240m_cmma_ddt_lag17', '240m_reactivity_dt', '80m_adx_14_dt', '240m_voss_filt_lag23', '80m_ac_20', '80m_natr_lag16', '240m_ma_difference_dt', '240m_kyle_lambda_ddt', '240m_ehlers_early_onset_trend_ddt', '240m_vwap_dt', '240m_price_variance_ratio_ddt', '240m_acc_swing_index_lag29', '240m_bandpass_ddt', '240m_hasbrouck_lambda_dt', '240m_adx_14_dt', '240m_acc_swing_index_lag21', '240m_adx_7_ddt', '240m_cmma', '240m_acc_swing_index_lag7', '240m_price_change_oscillator_dt', '240m_trendflex_dt', '240m_norm_on_balance_volume', '240m_acc_swing_index_lag8', '240m_hasbrouck_lambda_ddt', '240m_corwin_schultz_estimator', '80m_iqr_ratio_lag21', '80m_kyle_lambda_lag6', '240m_amihud_lambda_ddt', 'side_model_res', '720m_amihud_lambda_lag29', '240m_amihud_lam

In [4]:
import json

with open(f"data/meta_features_{selector.__class__.__name__}.json", "w") as f:
    json.dump(meta_res.to_dict(), f, indent=4)

In [5]:
import json

import pandas as pd

with open("data/meta_features_RFCQSelector.json", "r") as f:
    meta_res = pd.Series(json.load(f))

In [6]:
meta_res.to_dict()["side_model_res"]

0.01

In [7]:
from pathlib import Path
from strategies.BinanceMLV2.config import (
    DOLLAR_BAR_SHORT_TERM,
    DOLLAR_BAR_MID_TERM,
    DOLLAR_BAR_LONG_TERM,
)

feature_info = Path("strategies/BinanceMLV2/feature_info.json")
if feature_info.exists():
    with open(feature_info, "r") as f:
        feature_info = json.load(f)

feature_info["meta"][DOLLAR_BAR_SHORT_TERM] = []
feature_info["meta"][DOLLAR_BAR_MID_TERM] = []
feature_info["meta"][DOLLAR_BAR_LONG_TERM] = []
feature_info["meta"]["model_res"] = []

for k, v in meta_res[meta_res > 0].to_dict().items():
    if k.startswith(f"{DOLLAR_BAR_SHORT_TERM}_"):
        feature_info["meta"][DOLLAR_BAR_SHORT_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_MID_TERM}_"):
        feature_info["meta"][DOLLAR_BAR_MID_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_LONG_TERM}_"):
        feature_info["meta"][DOLLAR_BAR_LONG_TERM].append(k)
    else:
        feature_info["meta"]["model_res"].append(k)


feature_info["side"][DOLLAR_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_SHORT_TERM])),
)
feature_info["side"][DOLLAR_BAR_MID_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_MID_TERM])),
)
feature_info["side"][DOLLAR_BAR_LONG_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_LONG_TERM])),
)


feature_info["meta"][DOLLAR_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_SHORT_TERM])),
)
feature_info["meta"][DOLLAR_BAR_MID_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_MID_TERM])),
)
feature_info["meta"][DOLLAR_BAR_LONG_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_LONG_TERM])),
)
feature_info["meta"]["model_res"] = sorted(
    list(set(feature_info["meta"]["model_res"])),
)

with open("strategies/BinanceMLV2/feature_info.json", "w") as f:
    json.dump(feature_info, f, indent=4)